### Step6: prepare data as pickled dict for ELLA analysis

In [1]:
import numpy as np
import pandas as pd
import anndata
import matplotlib.pyplot as plt
import cv2
from sklearn.cluster import KMeans
import pickle
import timeit

/net/mulan/home/jadewang/tools/miniconda3/envs/torch/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Load data

In [5]:
##### read processed gene expression data (from step5)
outfile = 'output_step5/seqscope_df_dict.pkl'
# load
with open(outfile, 'rb') as f:
    pickle_dict = pickle.load(f)
df = pickle_dict['df']

df.head()

,geneID,tile,x,y,umi,cell,centerX,centerY,sc_total,sc_xmin,...,sc_xrange,sc_ymin,sc_ymax,sc_yrange,sc_umi,sc_umi_npc,sc_umi_pc,sc_umi_pp,tile_cell,type
157,Xkr4,2102,17721,19908,1,1720,17777,19511,2865,17328,...,809,19170,20021,851,1,5,16,203,2102_1720,-1
315,Xkr4,2102,16686,2244,1,1842,16808,2311,1226,16361,...,813,2100,2718,618,1,5,5,47,2102_1842,1
360,Xkr4,2102,18927,20667,1,1553,18932,20230,5703,18625,...,710,19840,20696,856,1,24,77,533,2102_1553,1
631,Xkr4,2102,19007,11672,1,1504,19294,11744,5016,18929,...,747,11340,12299,959,1,24,44,357,2102_1504,1
866,Xkr4,2102,21559,12652,1,1207,21496,12998,3848,21034,...,868,12611,13683,1072,1,8,21,262,2102_1207,-1


In [3]:
##### load masks for all tiles (from step3)
tileList = [2102, 2103, 2104, 2105, 2106, 2107, 2116, 2117, 2118, 2119]
masks_dict = {}

start = timeit.default_timer()

for t in tileList:
    print(t)
    outfile = 'output_step3/masks_4X_tile'+str(t)+'_dict.pkl'
    # load
    with open(outfile, 'rb') as f:
        pickle_dict = pickle.load(f)
    masks_t = pickle_dict['mask'+str(t)]
    # add
    masks_dict['mask'+str(t)] = masks_t
print(masks_dict.keys())

stop = timeit.default_timer()
print('Time: ', stop - start) # ~481s

2102
2103
2104
2105
2106
2107
2116
2117
2118
2119
dict_keys(['mask2102', 'mask2103', 'mask2104', 'mask2105', 'mask2106', 'mask2107', 'mask2116', 'mask2117', 'mask2118', 'mask2119'])
Time:  475.04225773969665


#### Prepare df 
gene expression data

In [6]:
##### 1. rename cols
df1 = df.rename(columns={'cell':'_cell', 'tile_cell':'cell', 'type':'_type', 'geneID':'gene'})
print('df1 #cells', df1.cell.nunique())

df1 #cells 995


In [7]:
##### 2. create a new 'type' col H012, TD012
tileList_H = [2102, 2103, 2104, 2105, 2106, 2107]
tileList_TD = [2116, 2117, 2118, 2119]
# add new type col H012 TD012
tile_num = df1.tile
type_old = df1._type
type_new = []
for it, t in enumerate(tile_num):
    if t in tileList_H:
        type_new.append('H'+str(type_old.iloc[it]))
    else:
        type_new.append('TD'+str(type_old.iloc[it]))
df1['type'] = np.array(type_new)

In [8]:
##### 3. only keep PC (H1, TD1) and PP (H2, TD2) cells
df3 = df1[df1.type.isin(['H1', 'H2', 'TD1', 'TD2'])].copy()
print('df3 #cells', df3.cell.nunique())

df3 #cells 895


In [6]:
##### 4. only keep cells with x/yrange $\in$ [500,1300] (approx the [0.05,0.95] quantiles)
f1 = df3.sc_xrange>=500
f2 = df3.sc_xrange<=1300
f3 = df3.sc_yrange>=500
f4 = df3.sc_yrange<=1300
df4 = df3[f1 & f2 & f3 & f4].copy()
print('df4 #cells', df4.cell.nunique())

df4 #cells 870


In [7]:
df4.head()

,gene,tile,x,y,umi,_cell,centerX,centerY,sc_total,sc_xmin,...,sc_ymin,sc_ymax,sc_yrange,sc_umi,sc_umi_npc,sc_umi_pc,sc_umi_pp,cell,_type,type
315,Xkr4,2102,16686,2244,1,1842,16808,2311,1226,16361,...,2100,2718,618,1,5,5,47,2102_1842,1,H1
360,Xkr4,2102,18927,20667,1,1553,18932,20230,5703,18625,...,19840,20696,856,1,24,77,533,2102_1553,1,H1
631,Xkr4,2102,19007,11672,1,1504,19294,11744,5016,18929,...,11340,12299,959,1,24,44,357,2102_1504,1,H1
985,Xkr4,2102,8396,15124,1,2966,8420,14913,3290,8125,...,14588,15606,1018,2,15,11,397,2102_2966,2,H2
1156,Xkr4,2102,16421,20213,1,1903,16262,20279,1520,15946,...,19868,20644,776,1,8,4,110,2102_1903,2,H2


#### Gene and cells lists for each type

In [9]:
##### cell type list
type_list = df4.type.unique().tolist()
type_list.sort()
print(type_list)

['H1', 'H2', 'TD1', 'TD2']


In [10]:
##### filtering params
# 1. min #cells available for a gene
nc_avl_min = 50 
# 2. >=`nc_expr_thre` number of cells with gene counts>=`expr_thre`
expr_thre = 3
nc_expr_thre = 5
# 3. add nuclear genes manually [skipped!]
# 4. add cell type marker genes [skipped!]
# 5. sort the genes and get the dataframe

In [11]:
# 1. nc_avl_min
gene_list_dict1 = {}
df_dict1 = {}
for t in type_list:
    df_t = df4[df4.type==t]
    nc_avl_t = df_t.groupby('gene')['cell'].nunique()
    nc_avl_t_filtered = nc_avl_t[nc_avl_t>=nc_avl_min]
    gene_list_dict1[t] = nc_avl_t_filtered.index.to_numpy()
    df_dict1[t] = df_t[df_t.gene.isin(gene_list_dict1[t])]
    print(f'type {t} #genes={len(gene_list_dict1[t])}')

type H1 #genes=2234
type H2 #genes=2186
type TD1 #genes=500
type TD2 #genes=2154


In [12]:
# 2. nc_expr_thre and expr_thre
gene_list_dict2 = {}
for t in type_list:
    df_t = df_dict1[t]
    df_t_gbG = df_t.groupby('gene')
    gl_t = []
    for g in gene_list_dict1[t][:]:
        df_t_g = df_t_gbG.get_group(g)
        sc_umi_g = df_t_g.groupby('cell')['umi'].sum()
        if np.sum(sc_umi_g>=expr_thre)>=nc_expr_thre:
            gl_t.append(g)
    gene_list_dict2[t] = np.array(gl_t)
    print(f'type {t} #genes={len(gl_t)}')

type H1 #genes=1295
type H2 #genes=1318
type TD1 #genes=497
type TD2 #genes=1349


In [13]:
# 5. get the df and sort the genes
gene_list_dict4 = gene_list_dict2 #!!! step 3/4 skipped

gene_list_dict5 = {}
cell_list_dict5 = {}
df_dict5 = {}
for t in type_list:
    gl_t4 = gene_list_dict4[t]
    df_t = df4[df4.type==t]
    df_t5 = df_t[df_t.gene.isin(gl_t4)]
    gl_t5 = df_t5.groupby('gene')['umi'].sum().sort_values(ascending=False).index.to_numpy()
    gene_list_dict5[t] = gl_t5
    df_dict5[t] = df_t5
    cell_list_dict5[t] = df_t5.cell.unique().tolist()

In [14]:
data_df = pd.concat(list(df_dict5.values()))
gene_list_dict = gene_list_dict5
cell_list_dict = cell_list_dict5
cell_list_all = [item for sublist in list(cell_list_dict.values()) for item in sublist]

#### Prepare cell masks

In [16]:
start = timeit.default_timer()

n_pad = 10 # pad for better cropping cell masks <<<<<
df_gyC = data_df.groupby('cell')

cell_masks_dict = {}

for ic, c in enumerate(cell_list_all):
    # tile and cell-only index
    t, c2 = c.split('_')
    
    # df for c
    df_c = df_gyC.get_group(c).copy()
    
    # crop mask for c
    mask_t = masks_dict['mask'+str(t)]
    xmin = df_c.sc_xmin.iloc[0]
    xmax = df_c.sc_xmax.iloc[0]
    ymin = df_c.sc_ymin.iloc[0]
    ymax = df_c.sc_ymax.iloc[0]
    mask_c = mask_t[(xmin-n_pad):(xmax+n_pad), (ymin-n_pad):(ymax+n_pad)]
    tuple_c = np.where(mask_c==int(c2))
    mask_df_c = pd.DataFrame({'x': (tuple_c[0]+(xmin-n_pad)), 'y': (tuple_c[1]+(ymin-n_pad))})
    mask_df_c['cell'] = [c]*len(mask_df_c)
    
    # add
    cell_masks_dict[c] = mask_df_c
    
# concatenate all cell_masks_dict to one df
cell_masks = pd.concat(list(cell_masks_dict.values()))

stop = timeit.default_timer()
print('Time: ', stop - start) # ~140s

Time:  67.9176314920187


In [17]:
cell_masks.head()

,x,y,cell
0,24546,23490,2102_686
1,24546,23491,2102_686
2,24546,23492,2102_686
3,24546,23493,2102_686
4,24546,23494,2102_686


In [18]:
len(cell_list_all)

870

#### Save prepared data

In [19]:
##### save data
outfile = 'output_step6/seqscope_data_dict.pkl'
# save
pickle_dict = {}
pickle_dict['type_list'] = type_list
pickle_dict['gene_list_dict'] = gene_list_dict
pickle_dict['cell_list_dict'] = cell_list_dict
pickle_dict['cell_list_all'] = cell_list_all
pickle_dict['cell_mask_df'] = cell_masks
pickle_dict['data_df'] = data_df
with open(outfile, 'wb') as f:
    pickle.dump(pickle_dict, f)

# load
# with open(outfile, 'rb') as f:
#     pickle_dict = pickle.load(f)
# registered_dict = pickle_dict['df_registered']